# Combine Exposures

- The following document combines multiple exposures of a single object by taking their median 
- Specifically, for exposures of the same object taken at the same time with the GHOST spectrograph
    - The exposures should have been taken at the same time so that flux does not change significanlty between them
        - This means that they do not need to be continuum normalized before this stage  
    - The exposures should have the same wavelength array (at this time, this is the case for GHOST spectra)
    
- Modifications can be made if other functionality is needed

In [1]:
import numpy as np
import os 
import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import itertools
import sys

%matplotlib notebook

# ----------------- Import the other files of functions
module_path = os.path.join('/arc5/home/jglover/ASAP')
if module_path not in sys.path:
    sys.path.append(module_path)
    
import asap_lib.spectra as sa

In [3]:
extensionVersion = 'new' # CHANGE ME

IFU = 0 # CHANGE ME

# Name of your object 
star = 'Spec_phot' # CHANGE ME

# Path to folder containing formatted output files for JUST that obejct
path = '../GHOST_ObsData/{}/output/'.format(star)# CHANGE ME

spath = path+star

In [4]:
# ---------- There should be (theoretically) nothing to change in the following cell 
for camera in ['blue', 'red']:

    condition = '*{}*formattedOutput.fits'.format(camera)
    obs = glob.glob(path+condition)
    
    print('Found {} exposures to combine: '.format(len(obs)))
    print(*obs, sep="\n")

    # --------------------------------
    waves = []
    fluxes = []
    errs = []

    for item in obs:
        spectra = fits.open(item)

        if extensionVersion == 'old':
            wave = spectra[7].data
            flux = spectra[5].data[:,IFU]
            err = spectra[6].data[:,IFU]
            
        if extensionVersion == 'new':
            wave = spectra[11].data
            flux = spectra[12].data[:,IFU]
            err = spectra[13].data[:,IFU]

        waves.append(wave)
        fluxes.append(flux)
        errs.append(err)

    # --------------------------------
    pairs =  list(itertools.combinations(range(len(waves)), 2))

    for pair in pairs:
        value = np.array_equal(waves[pair[0]], waves[pair[1]])
        if value == False:
            print('Wavelength arrays not equivalent. Do not proceed')
            break

    if len(waves) > 0:
        # -------------------------------- Median Combine flux and combine error with Poisson statistics 
        finalWave = waves[0]
        finalFlux = np.median(fluxes,axis=0)
        finalErr = np.sqrt( sum(i*i for i in errs) ) / len(errs) 


        # -------------------------------- Save the output as .xy and .bin files 
        sa.write2xy(finalWave, finalFlux, finalErr, spath+'_{}'.format(camera))
        sa.write2bin(finalWave, finalFlux, finalErr, spath+'_{}'.format(camera))

        print('Saved combination of {} {} camera images'.format(len(waves), camera))
        # --------------------------------

    else:
        print('No {} camera files found'.format(camera))

Saved combination of 3 blue camera images
Saved combination of 1 red camera images
